# Linear Programming with PuLP
## 1. 分析问题，定义已知量

**目标**: 以满足各个商店需求为前提，设计一个总花费最小的方案，包括:
- 决定是否在某地建设工厂
- 需要从A工厂运输到B商店的货物量

**已知**:
- 四个场地的产能（supply）和建设费用（fixedCost）
- 四个商店的需求（demand）
- 从A工厂运输到B商店的单位运费（costs）

In [1]:
plants = ['Dallas', 'Los Angeles', 'Seattle', 'Denver']
supply = {'Dallas':900, 'Los Angeles':2400, 'Seattle':1300, 'Denver':1800}
fixedCost = {'Dallas':75000, 'Los Angeles':72000, 'Seattle':100000, 'Denver':74000}

In [2]:
stores = ['Austin', 'Sacaramento', 'Philadelphia', 'Boulder']
demand = {'Austin':1700, 'Sacaramento':1000, 'Philadelphia':1500, 'Boulder':1200}

In [3]:
cost_matrix = [
    #ATX #SAC #PHL #BU
    [2,   7,   4,   6], #DAL
    [6,   3,   4,   5], #LA
    [8,   4,   6,   5], #SEA
    [7,   6,   5,   1]  #DEN
]
costs = {plants[i]:{stores[j]:cost_matrix[i][j] for j in range(len(stores))} for i in range(len(plants))}
costs

{'Dallas': {'Austin': 2, 'Sacaramento': 7, 'Philadelphia': 4, 'Boulder': 6},
 'Los Angeles': {'Austin': 6,
  'Sacaramento': 3,
  'Philadelphia': 4,
  'Boulder': 5},
 'Seattle': {'Austin': 8, 'Sacaramento': 4, 'Philadelphia': 6, 'Boulder': 5},
 'Denver': {'Austin': 7, 'Sacaramento': 6, 'Philadelphia': 5, 'Boulder': 1}}

In [4]:
# 定义路径来表示所有可能的运输情况
routes = []
for p in plants:
    for s in stores:
        routes.append((p,s))
routes

[('Dallas', 'Austin'),
 ('Dallas', 'Sacaramento'),
 ('Dallas', 'Philadelphia'),
 ('Dallas', 'Boulder'),
 ('Los Angeles', 'Austin'),
 ('Los Angeles', 'Sacaramento'),
 ('Los Angeles', 'Philadelphia'),
 ('Los Angeles', 'Boulder'),
 ('Seattle', 'Austin'),
 ('Seattle', 'Sacaramento'),
 ('Seattle', 'Philadelphia'),
 ('Seattle', 'Boulder'),
 ('Denver', 'Austin'),
 ('Denver', 'Sacaramento'),
 ('Denver', 'Philadelphia'),
 ('Denver', 'Boulder')]

## 2. 构建线性规划模型

In [5]:
import pulp

### 2.1 优化变量
```py
pulp.LpVariable.dicts("变量名", 变量_indices, lowBound, upperBound, 数据类型(整/浮点))
```

例如，如下的:
- `trans` 定义了16个表示从A工厂运输到B商店的货物量（Transport_A_B）(>=0)
- `build` 定义了4个表示A工厂是否建设的变量（Build_Plants_A）(=0 or 1)


In [6]:
trans = pulp.LpVariable.dicts("Transport", (plants,stores), 0, None, pulp.LpInteger)
build = pulp.LpVariable.dicts("Build_Plants", plants, 0, 1, pulp.LpInteger)
trans

{'Dallas': {'Austin': Transport_Dallas_Austin,
  'Sacaramento': Transport_Dallas_Sacaramento,
  'Philadelphia': Transport_Dallas_Philadelphia,
  'Boulder': Transport_Dallas_Boulder},
 'Los Angeles': {'Austin': Transport_Los_Angeles_Austin,
  'Sacaramento': Transport_Los_Angeles_Sacaramento,
  'Philadelphia': Transport_Los_Angeles_Philadelphia,
  'Boulder': Transport_Los_Angeles_Boulder},
 'Seattle': {'Austin': Transport_Seattle_Austin,
  'Sacaramento': Transport_Seattle_Sacaramento,
  'Philadelphia': Transport_Seattle_Philadelphia,
  'Boulder': Transport_Seattle_Boulder},
 'Denver': {'Austin': Transport_Denver_Austin,
  'Sacaramento': Transport_Denver_Sacaramento,
  'Philadelphia': Transport_Denver_Philadelphia,
  'Boulder': Transport_Denver_Boulder}}

### 2.2 优化目标

优化目标: 最小化费用 = 工厂建设费 + 运费

In [7]:
obj = ""

for (p,s) in routes:
    obj += trans[p][s] * costs[p][s]
    
for p in plants:
    obj += fixedCost[p] * build[p]

In [8]:
prob = pulp.LpProblem("Supply_Demand", pulp.LpMinimize)
prob += obj, 'Total Cost'
prob

Supply_Demand:
MINIMIZE
75000*Build_Plants_Dallas + 74000*Build_Plants_Denver + 72000*Build_Plants_Los_Angeles + 100000*Build_Plants_Seattle + 2*Transport_Dallas_Austin + 6*Transport_Dallas_Boulder + 4*Transport_Dallas_Philadelphia + 7*Transport_Dallas_Sacaramento + 7*Transport_Denver_Austin + 1*Transport_Denver_Boulder + 5*Transport_Denver_Philadelphia + 6*Transport_Denver_Sacaramento + 6*Transport_Los_Angeles_Austin + 5*Transport_Los_Angeles_Boulder + 4*Transport_Los_Angeles_Philadelphia + 3*Transport_Los_Angeles_Sacaramento + 8*Transport_Seattle_Austin + 5*Transport_Seattle_Boulder + 6*Transport_Seattle_Philadelphia + 4*Transport_Seattle_Sacaramento + 0
VARIABLES
0 <= Build_Plants_Dallas <= 1 Integer
0 <= Build_Plants_Denver <= 1 Integer
0 <= Build_Plants_Los_Angeles <= 1 Integer
0 <= Build_Plants_Seattle <= 1 Integer
0 <= Transport_Dallas_Austin Integer
0 <= Transport_Dallas_Boulder Integer
0 <= Transport_Dallas_Philadelphia Integer
0 <= Transport_Dallas_Sacaramento Integer
0 <= Tr

### 2.4 约束 Constraints

主要有两个约束
- 从工厂运出的货物量不能超过其产能
- 商店接受的货物量不能低于其需求

In [9]:
for p in plants:
    product_out = ""
    for s in stores:
        product_out += trans[p][s]
    prob += product_out <= supply[p] * build[p], 'Total product out of plant_'+str(p)
prob

Supply_Demand:
MINIMIZE
75000*Build_Plants_Dallas + 74000*Build_Plants_Denver + 72000*Build_Plants_Los_Angeles + 100000*Build_Plants_Seattle + 2*Transport_Dallas_Austin + 6*Transport_Dallas_Boulder + 4*Transport_Dallas_Philadelphia + 7*Transport_Dallas_Sacaramento + 7*Transport_Denver_Austin + 1*Transport_Denver_Boulder + 5*Transport_Denver_Philadelphia + 6*Transport_Denver_Sacaramento + 6*Transport_Los_Angeles_Austin + 5*Transport_Los_Angeles_Boulder + 4*Transport_Los_Angeles_Philadelphia + 3*Transport_Los_Angeles_Sacaramento + 8*Transport_Seattle_Austin + 5*Transport_Seattle_Boulder + 6*Transport_Seattle_Philadelphia + 4*Transport_Seattle_Sacaramento + 0
SUBJECT TO
Total_product_out_of_plant_Dallas: - 900 Build_Plants_Dallas
 + Transport_Dallas_Austin + Transport_Dallas_Boulder
 + Transport_Dallas_Philadelphia + Transport_Dallas_Sacaramento <= 0

Total_product_out_of_plant_Los_Angeles: - 2400 Build_Plants_Los_Angeles
 + Transport_Los_Angeles_Austin + Transport_Los_Angeles_Boulder
 + 

In [10]:
for s in stores:
    product_in = ""
    for p in plants:
        product_in += trans[p][s]
    prob += product_in >= demand[s], 'Total product into store_'+str(s)
prob

Supply_Demand:
MINIMIZE
75000*Build_Plants_Dallas + 74000*Build_Plants_Denver + 72000*Build_Plants_Los_Angeles + 100000*Build_Plants_Seattle + 2*Transport_Dallas_Austin + 6*Transport_Dallas_Boulder + 4*Transport_Dallas_Philadelphia + 7*Transport_Dallas_Sacaramento + 7*Transport_Denver_Austin + 1*Transport_Denver_Boulder + 5*Transport_Denver_Philadelphia + 6*Transport_Denver_Sacaramento + 6*Transport_Los_Angeles_Austin + 5*Transport_Los_Angeles_Boulder + 4*Transport_Los_Angeles_Philadelphia + 3*Transport_Los_Angeles_Sacaramento + 8*Transport_Seattle_Austin + 5*Transport_Seattle_Boulder + 6*Transport_Seattle_Philadelphia + 4*Transport_Seattle_Sacaramento + 0
SUBJECT TO
Total_product_out_of_plant_Dallas: - 900 Build_Plants_Dallas
 + Transport_Dallas_Austin + Transport_Dallas_Boulder
 + Transport_Dallas_Philadelphia + Transport_Dallas_Sacaramento <= 0

Total_product_out_of_plant_Los_Angeles: - 2400 Build_Plants_Los_Angeles
 + Transport_Los_Angeles_Austin + Transport_Los_Angeles_Boulder
 + 

### 2.5 求解规划问题

In [11]:
# 求解各变量的值
prob.solve()
print("Status: ", pulp.LpStatus[prob.status])
for v in prob.variables():
    print(v.name, '=', v.varValue)

Status:  Optimal
Build_Plants_Dallas = 0.0
Build_Plants_Denver = 1.0
Build_Plants_Los_Angeles = 1.0
Build_Plants_Seattle = 1.0
Transport_Dallas_Austin = 0.0
Transport_Dallas_Boulder = 0.0
Transport_Dallas_Philadelphia = 0.0
Transport_Dallas_Sacaramento = 0.0
Transport_Denver_Austin = 600.0
Transport_Denver_Boulder = 1200.0
Transport_Denver_Philadelphia = 0.0
Transport_Denver_Sacaramento = 0.0
Transport_Los_Angeles_Austin = 900.0
Transport_Los_Angeles_Boulder = 0.0
Transport_Los_Angeles_Philadelphia = 1500.0
Transport_Los_Angeles_Sacaramento = 0.0
Transport_Seattle_Austin = 200.0
Transport_Seattle_Boulder = 0.0
Transport_Seattle_Philadelphia = 0.0
Transport_Seattle_Sacaramento = 1000.0


In [12]:
# 求解目标优化值
pulp.value(prob.objective)

268400.0